<a href="https://colab.research.google.com/github/Luciesprogram/Plant-Disease-Recognition-using-CNN/blob/main/Plant_Disease_Recognition_down.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset Load

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rashikrahmanpritom/plant-disease-recognition-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1


## Importing Libraries

In [ ]:
# Step 1: Import Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


## Spliting Dataset into train, test and split datasets

In [ ]:
# Step 3: Set dataset directories
train_dir = os.path.join(path, "Train/Train")
valid_dir = os.path.join(path, "Validation")
test_dir = os.path.join(path, "Test/Test")

print("Train path:", train_dir)
print("Validation path:", valid_dir)
print("Test path:", test_dir)

# Verify classes
print("Classes:", os.listdir(train_dir))

Train path: /root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Train/Train
Validation path: /root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Validation
Test path: /root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Test/Test
Classes: ['Rust', 'Powdery', 'Healthy']


## Scaling the images

In [ ]:

# Image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Data generators without augmentation for validation and testing
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False) # Keep data in order for evaluation

# Get class labels
class_labels = list(train_generator.class_indices.keys())
print("Class labels:", class_labels)
num_classes = len(class_labels)
print("Number of classes:", num_classes)

Found 1322 images belonging to 3 classes.
Found 60 images belonging to 1 classes.
Found 150 images belonging to 3 classes.
Class labels: ['Healthy', 'Powdery', 'Rust']
Number of classes: 3


In [ ]:
# Step 5: Display sample images from each class

# Get class labels
class_labels = list(train_generator.class_indices.keys())
print("Class labels:", class_labels)
num_classes = len(class_labels)
print("Number of classes:", num_classes)



Class labels: ['Healthy', 'Powdery', 'Rust']
Number of classes: 3


In [ ]:

true_classes_train = train_generator.classes


class_counts_train = np.bincount(true_classes_train)


class_indices_train = train_generator.class_indices
class_names_train = dict((v, k) for k, v in class_indices_train.items())


print("Number of images per class in the training set:")
for class_index, count in enumerate(class_counts_train):
    class_name = class_names_train[class_index]
    print(f"{class_name}: {count}")

Number of images per class in the training set:
Healthy: 458
Powdery: 430
Rust: 434


## Define the model

## Creating a sequential model and adding layers in sequential modek.

In [ ]:
# Step 6: Build the CNN model

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,035,203 (72.61 MB)

 Trainable params: 19,035,203 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

## Compile the model




Compile the defined model with the specified optimizer, loss function, and metrics.



In [ ]:
# Step 7: Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,035,203 (72.61 MB)

 Trainable params: 19,035,203 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

## Train the model



In [ ]:
# Image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # Split training data for validation

# Data generators without augmentation for testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow images from directories
print("Total images for training")
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # Set as training subset

print("Total images for validation")
valid_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical', # Changed to 'categorical'
    subset='validation') # Set as validation subset

print("Total images for testing")
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False) # Keep data in order for evaluation

# Get class labels
class_labels = list(train_generator.class_indices.keys())
print("Class labels:", class_labels)
num_classes = len(class_labels)
print("Number of classes:", num_classes)


epochs = 10

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    callbacks=[early_stopping]
)

Found 1059 images belonging to 3 classes.
Found 263 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Class labels: ['Healthy', 'Powdery', 'Rust']
Number of classes: 3


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.6766 - loss: 0.6886 - val_accuracy: 0.7383 - val_loss: 0.5379
Epoch 2/10
 1/33 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.8438 - loss: 0.4451

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


33/33 ━━━━━━━━━━━━━━━━━━━━ 27s 805ms/step - accuracy: 0.8438 - loss: 0.4451 - val_accuracy: 0.7969 - val_loss: 0.4946
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 196s 6s/step - accuracy: 0.8178 - loss: 0.4906 - val_accuracy: 0.7578 - val_loss: 0.5619
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 630ms/step - accuracy: 0.8438 - loss: 0.4736 - val_accuracy: 0.8047 - val_loss: 0.4905
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 139s 4s/step - accuracy: 0.8789 - loss: 0.3448 - val_accuracy: 0.8672 - val_loss: 0.3346
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 22s 626ms/step - accuracy: 0.9688 - loss: 0.1911 - val_accuracy: 0.8516 - val_loss: 0.3724
Epoch 7/10
 3/33 ━━━━━━━━━━━━━━━━━━━━ 2:18 5s/step - accuracy: 0.8611 - loss: 0.3164

KeyboardInterrupt: 

In [ ]:
# Step 10: Visualize training history

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')

plt.show()

## Evaluate the model


In [ ]:
# Step 9: Evaluate the model on the test set
print("Evaluating the model on the test set...")
test_loss, test_acc = model.evaluate(test_generator)

print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Geting predictions on the test set
print("\nGetting predictions on the test set...")
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = test_generator.classes
class_indices = test_generator.class_indices
# Invert the dictionary to get class names from indices
class_names = dict((v, k) for k, v in class_indices.items())
true_class_names = [class_names[i] for i in true_classes]
predicted_class_names = [class_names[i] for i in predicted_classes]


# Generate classification report
print("\nClassification Report:")
print(classification_report(true_class_names, predicted_class_names))

# Generate confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
from sklearn.metrics import f1_score

# Calculate F1 score for each class
f1_scores = f1_score(true_classes, predicted_classes, average=None)


weighted_f1_score = f1_score(true_classes, predicted_classes, average='weighted')

print("F1 Score per class:", f1_scores)

## Classify and visualize test images

In [ ]:


# Get class names from indices
class_indices = test_generator.class_indices
class_names = dict((v, k) for k, v in class_indices.items())

# Dictionary to store the first image found for each class
images_to_display = {}

# Iterate through the test generator to find one image from each class
for i in range(len(test_generator)):
    test_images, test_labels = test_generator[i]
    predictions = model.predict(test_images)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(test_labels, axis=1)

    for j in range(len(test_images)):
        true_class_name = class_names[true_classes[j]]
        predicted_class_name = class_names[predicted_classes[j]]

        if true_class_name not in images_to_display:
            images_to_display[true_class_name] = {
                'image': test_images[j],
                'true_label': true_class_name,
                'predicted_label': predicted_class_name
            }

        # Stop once we have found an image for each class
        if len(images_to_display) == len(class_names):
            break
    if len(images_to_display) == len(class_names):
        break


# Display the first image found for each class (up to 3)
plt.figure(figsize=(15, 5))
for i, (class_name, image_info) in enumerate(images_to_display.items()):
    if i < 3:
        plt.subplot(1, 3, i + 1)
        plt.imshow(image_info['image'])
        plt.title(f"True: {image_info['true_label']}\nPred: {image_info['predicted_label']}")
        plt.axis('off')
plt.tight_layout()
plt.show()